In [ ]:
# TEMPORARY TABLE (QUERY PERTAMA)

WITH report_monthly_orders_product_agg AS (
    SELECT
        p.id AS product_id,
        p.name AS product_name,
        p.category AS product_category,
        p.brand AS product_brand,
        EXTRACT(YEAR FROM oi.delivered_at) AS order_year,  
        EXTRACT(MONTH FROM oi.delivered_at) AS order_month, 
        SUM(oi.sale_price) AS total_sales,
        COUNT(oi.id) AS total_items_sold, 
        SUM(oi.sale_price) / COUNT(oi.id) AS average_price_per_item 
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items` oi
    JOIN
        `bigquery-public-data.thelook_ecommerce.products` p
        ON oi.product_id = p.id
    JOIN
        `bigquery-public-data.thelook_ecommerce.orders` o
        ON oi.order_id = o.order_id
    WHERE
        oi.delivered_at IS NOT NULL 
    GROUP BY
        p.id, p.name, p.category, p.brand, order_year, order_month
)

# QUERY UTAMA

SELECT
    product_id,
    product_name,
    product_category,
    product_brand,
    order_year,
    order_month,
    total_sales,
    total_items_sold,
    average_price_per_item
FROM
    report_monthly_orders_product_agg
ORDER BY
    order_year DESC,
    order_month DESC,
    total_sales DESC; 

**Penjelasan:**

- **WITH report_monthly_orders_product_agg AS (...):**  
  Bagian ini mendefinisikan sebuah Common Table Expression (CTE) yang diberi nama `report_monthly_orders_product_agg`. CTE digunakan untuk membuat query sementara yang dapat dipanggil kembali dalam bagian query utama. Dalam hal ini, CTE ini akan menghitung agregat penjualan produk per bulan dan per tahun.

- **SELECT:**  
  Query ini memilih berbagai kolom untuk informasi produk dan agregat penjualan. Kolom-kolom yang dipilih adalah:
  - product_id: ID produk dari tabel `products`.
  - product_name: Nama produk dari tabel `products`.
  - product_category: Kategori produk dari tabel `products`.
  - product_brand: Merek produk dari tabel `products`.
  - order_year: Tahun pengiriman pesanan yang diekstrak dari tanggal pengiriman.
  - order_month: Bulan pengiriman pesanan yang diekstrak dari tanggal pengiriman.
  - total_sales: Jumlah total penjualan (penjumlahan harga jual dari semua item yang terjual).
  - total_items_sold: Jumlah total item yang terjual.
  - average_price_per_item: Harga rata-rata per item (total penjualan dibagi dengan jumlah item yang terjual).

- **FROM bigquery-public-data.thelook_ecommerce.order_items oi:**  
  Query ini mengambil data dari tabel `order_items` yang berada dalam dataset `thelook_ecommerce` yang disediakan oleh Google BigQuery. Tabel ini berisi informasi tentang item yang dipesan dalam setiap transaksi.

- **JOIN bigquery-public-data.thelook_ecommerce.products p ON oi.product_id = p.id:**  
  Tabel `order_items` di-join dengan tabel `products` berdasarkan `product_id` untuk mendapatkan informasi produk terkait dengan setiap item yang dipesan. Kolom produk seperti nama, kategori, dan merek diambil dari tabel `products`.

- **JOIN bigquery-public-data.thelook_ecommerce.orders o ON oi.order_id = o.order_id:**  
  Tabel `order_items` juga di-join dengan tabel `orders` berdasarkan `order_id`. Ini digunakan untuk mengaitkan item yang terjual dengan informasi pesanan yang lebih besar.

- **WHERE oi.delivered_at IS NOT NULL:**  
  Kondisi ini memastikan bahwa hanya item pesanan yang sudah dikirim (dengan nilai `delivered_at` yang tidak kosong) yang dihitung dalam agregat.

- **GROUP BY:**  
  Kolom-kolom yang digunakan dalam **GROUP BY** adalah `p.id`, `p.name`, `p.category`, `p.brand`, `order_year`, dan `order_month`. Ini memastikan bahwa agregat dihitung per produk dan per bulan, dan mengelompokkan data berdasarkan ID produk, nama produk, kategori, merek, serta tahun dan bulan pengiriman.

- **SELECT (Main Query):**  
  Setelah CTE selesai didefinisikan, query utama memilih data dari CTE `report_monthly_orders_product_agg`. Kolom yang dipilih adalah yang sudah dihitung dalam CTE sebelumnya, yaitu ID produk, nama produk, kategori, merek, tahun dan bulan pesanan, total penjualan, total item yang terjual, dan harga rata-rata per item.

- **ORDER BY:**  
  Bagian ini mengurutkan hasil berdasarkan:
  - order_year DESC: Tahun pesanan diurutkan dari yang terbaru ke yang paling lama.
  - order_month DESC: Bulan pesanan diurutkan dari yang terbaru ke yang paling lama.
  - total_sales DESC: Total penjualan diurutkan dari yang tertinggi ke yang terendah.